# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [53]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

First version of our momentum screener!


In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [9]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2347636647260,
 'week52high': 146.56,
 'week52low': 72.13,
 'week52highSplitAdjustOnly': 146.94,
 'week52lowSplitAdjustOnly': 71.25,
 'week52change': 0.944481960059399,
 'sharesOutstanding': 17224632552,
 'float': 0,
 'avg10Volume': 84308423,
 'avg30Volume': 89130175,
 'day200MovingAvg': 126.22,
 'day50MovingAvg': 130.32,
 'employees': 147451,
 'ttmEPS': 3.74,
 'ttmDividendRate': 0.8588659963450721,
 'dividendYield': 0.006133420113331996,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-31',
 'nextEarningsDate': '2021-04-17',
 'peRatio': 36.40231891090434,
 'beta': 1.3287173430251227,
 'maxChangePercent': 52.3134765719199,
 'year5ChangePercent': 4.596228229670593,
 'year2ChangePercent': 1.7398465589210736,
 'year1ChangePercent': 0.948742781123622,
 'ytdChangePercent': 0.0173311572664894,
 'month6ChangePercent': 0.1789732179118661,
 'month3ChangePercent': -0.05821252336424237,
 'month1ChangePercent': 0.11507347380589254,
 'day30ChangePerce

In [10]:
data['year1ChangePercent']

0.948742781123622

## Executing A Batch API Call & Building Our DataFrame


In [11]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [12]:
df = pd.DataFrame(columns = my_columns)

In [18]:
for symbol in symbol_strings:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_url).json()
    for sym in symbol.split(','):
        df = df.append(
        pd.Series(
        [   sym,
            data[sym]['price'],
            data[sym]['stats']['year1ChangePercent'],
            'N/A'
        ], index = my_columns),ignore_index= True
        )
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,140.44,0.836303,N/A
1,AAL,22.78,1.18115,N/A
2,AAP,204.93,0.720653,N/A
3,AAPL,139.77,0.947156,N/A
4,ABBV,116.30,0.419516,N/A
...,...,...,...,...
500,YUM,121.51,0.386761,N/A
501,ZBH,178.92,0.531321,N/A
502,ZBRA,518.61,1.43289,N/A
503,ZION,57.65,0.937501,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold.

In [20]:
df = df.sort_values(by=['One-Year Price Return'], ascending=False)
df = df[:50]
df = df.reset_index()

## Calculating the Number of Shares to Buy

In [108]:
def portfolio_size():
    
    global portfolio_size
    portfolio_size = input('portfolio value: ')
    try:
        val  =float(portfolio_size)
    except ValueError:
        portfolio_size = input('Enter an integer: ')

In [40]:
portfolio_size()


portfolio value: 10000000


In [43]:
position_size = float(portfolio_size)/len(df.index)
for stock in range(0,len(df.index)):
    df.loc[stock, 'Number of Shares to Buy'] = math.floor(position_size /df.loc[stock, 'Price'] )

In [44]:
df

,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,275,LB,67.88,5.58052,2946
1,1,208,GPS,34.60,3.95442,5780
2,2,179,FCX,39.57,3.63576,5054
3,3,272,KSS,62.52,2.78144,3198
4,4,253,IVZ,27.69,2.62137,7222
5,5,441,TPR,48.56,2.47983,4118
6,6,24,ALGN,625.87,2.29944,319
7,7,410,SIVB,586.30,2.21757,341
8,8,314,MOS,36.73,2.15583,5445
9,9,462,URI,335.31,2.12784,596


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns


In [121]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_df = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df= hqm_df.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_df.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [125]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,136.85,N/A,0.8584,N/A,0.322979,N/A,0.114131,N/A,0.0949436,N/A,N/A
1,AAL,22.27,N/A,1.15919,N/A,0.89194,N/A,0.420919,N/A,-0.0391899,N/A,N/A
2,AAP,206.37,N/A,0.72035,N/A,0.312313,N/A,0.25303,N/A,0.0542521,N/A,N/A
3,AAPL,137.81,N/A,0.960362,N/A,0.174912,N/A,-0.0602298,N/A,0.113537,N/A,N/A
4,ABBV,112.15,N/A,0.408959,N/A,0.363793,N/A,0.0379355,N/A,0.0652982,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.03,N/A,0.379798,N/A,0.180638,N/A,0.0956184,N/A,0.0733581,N/A,N/A
501,ZBH,178.45,N/A,0.537109,N/A,0.268607,N/A,0.11439,N/A,0.0936695,N/A,N/A
502,ZBRA,519.32,N/A,1.44719,N/A,0.721918,N/A,0.268661,N/A,0.0463655,N/A,N/A
503,ZION,56.41,N/A,0.959542,N/A,0.723791,N/A,0.196359,N/A,-0.0222721,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`


In [123]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_df.index:
    for time_period in time_periods:
        if hqm_df.loc[row, f'{time_period} Price Return'] == None:
            hqm_df.loc[row, f'{time_period} Price Return'] = 0

In [126]:
for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])/100


In [127]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,136.85,N/A,0.8584,0.69703,0.322979,0.538614,0.114131,0.415842,0.0949436,0.859406,N/A
1,AAL,22.27,N/A,1.15919,0.849505,0.89194,0.946535,0.420919,0.980198,-0.0391899,0.0534653,N/A
2,AAP,206.37,N/A,0.72035,0.60198,0.312313,0.520792,0.25303,0.849505,0.0542521,0.631683,N/A
3,AAPL,137.81,N/A,0.960362,0.764356,0.174912,0.30297,-0.0602298,0.0435644,0.113537,0.914851,N/A
4,ABBV,112.15,N/A,0.408959,0.338614,0.363793,0.594059,0.0379355,0.192079,0.0652982,0.706931,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.03,N/A,0.379798,0.30495,0.180638,0.314851,0.0956184,0.338614,0.0733581,0.758416,N/A
501,ZBH,178.45,N/A,0.537109,0.469307,0.268607,0.451485,0.11439,0.417822,0.0936695,0.853465,N/A
502,ZBRA,519.32,N/A,1.44719,0.928713,0.721918,0.885149,0.268661,0.863366,0.0463655,0.548515,N/A
503,ZION,56.41,N/A,0.959542,0.760396,0.723791,0.887129,0.196359,0.69703,-0.0222721,0.0891089,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.


In [128]:
from statistics import mean

In [129]:
for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f"{time_period} Return Percentile"])
    hqm_df.loc[row, "HQM Score"] = mean(momentum_percentiles)

In [130]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,136.85,N/A,0.8584,0.69703,0.322979,0.538614,0.114131,0.415842,0.0949436,0.859406,0.627723
1,AAL,22.27,N/A,1.15919,0.849505,0.89194,0.946535,0.420919,0.980198,-0.0391899,0.0534653,0.707426
2,AAP,206.37,N/A,0.72035,0.60198,0.312313,0.520792,0.25303,0.849505,0.0542521,0.631683,0.65099
3,AAPL,137.81,N/A,0.960362,0.764356,0.174912,0.30297,-0.0602298,0.0435644,0.113537,0.914851,0.506436
4,ABBV,112.15,N/A,0.408959,0.338614,0.363793,0.594059,0.0379355,0.192079,0.0652982,0.706931,0.457921
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.03,N/A,0.379798,0.30495,0.180638,0.314851,0.0956184,0.338614,0.0733581,0.758416,0.429208
501,ZBH,178.45,N/A,0.537109,0.469307,0.268607,0.451485,0.11439,0.417822,0.0936695,0.853465,0.54802
502,ZBRA,519.32,N/A,1.44719,0.928713,0.721918,0.885149,0.268661,0.863366,0.0463655,0.548515,0.806436
503,ZION,56.41,N/A,0.959542,0.760396,0.723791,0.887129,0.196359,0.69703,-0.0222721,0.0891089,0.608416


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [131]:
hqm_df.sort_values(by= 'HQM Score', ascending = False, inplace=True)
hqm_df.reset_index(inplace=True, drop= True)

In [133]:
hqm_df = hqm_df[:50]

In [134]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,38.95,N/A,3.78463,0.99604,1.295,0.986139,0.467571,0.992079,0.188501,0.986139,0.990099
1,TPR,48.74,N/A,2.47731,0.990099,1.2108,0.984158,0.420876,0.978218,0.10317,0.883168,0.958911
2,GPS,35.20,N/A,3.91966,0.99802,0.638356,0.831683,0.562796,0.99802,0.144928,0.962376,0.947525
3,LB,67.95,N/A,5.49305,1,0.991141,0.974257,0.570232,1,0.0813447,0.805941,0.94505
4,MGM,42.30,N/A,2.05082,0.974257,0.902211,0.954455,0.407177,0.972277,0.102302,0.877228,0.944554
5,STX,96.28,N/A,1.02659,0.794059,0.948899,0.964356,0.465578,0.990099,0.23014,0.99802,0.936634
6,MOS,36.59,N/A,2.10551,0.982178,0.932938,0.960396,0.277671,0.877228,0.116885,0.918812,0.934653
7,KSU,302.27,N/A,1.34189,0.90495,0.695227,0.863366,0.443702,0.984158,0.186603,0.984158,0.934158
8,NCLH,30.60,N/A,1.83784,0.968317,0.815113,0.912871,0.28026,0.881188,0.129762,0.942574,0.926238
9,HPQ,35.67,N/A,1.37129,0.910891,0.94311,0.962376,0.414198,0.974257,0.0876998,0.837624,0.921287


## Calculating the Number of Shares to Buy

In [109]:
portfolio_size()

portfolio value: 1000000


In [135]:
position_size = float(portfolio_size)/len(df.index)
for stock in range(0,len(hqm_df.index)):
    hqm_df.loc[stock, 'Number of Shares to Buy'] = math.floor(position_size /hqm_df.loc[stock, 'Price'] )

In [136]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,38.95,513,3.78463,0.99604,1.295,0.986139,0.467571,0.992079,0.188501,0.986139,0.990099
1,TPR,48.74,410,2.47731,0.990099,1.2108,0.984158,0.420876,0.978218,0.10317,0.883168,0.958911
2,GPS,35.20,568,3.91966,0.99802,0.638356,0.831683,0.562796,0.99802,0.144928,0.962376,0.947525
3,LB,67.95,294,5.49305,1,0.991141,0.974257,0.570232,1,0.0813447,0.805941,0.94505
4,MGM,42.30,472,2.05082,0.974257,0.902211,0.954455,0.407177,0.972277,0.102302,0.877228,0.944554
5,STX,96.28,207,1.02659,0.794059,0.948899,0.964356,0.465578,0.990099,0.23014,0.99802,0.936634
6,MOS,36.59,546,2.10551,0.982178,0.932938,0.960396,0.277671,0.877228,0.116885,0.918812,0.934653
7,KSU,302.27,66,1.34189,0.90495,0.695227,0.863366,0.443702,0.984158,0.186603,0.984158,0.934158
8,NCLH,30.60,653,1.83784,0.968317,0.815113,0.912871,0.28026,0.881188,0.129762,0.942574,0.926238
9,HPQ,35.67,560,1.37129,0.910891,0.94311,0.962376,0.414198,0.974257,0.0876998,0.837624,0.921287
